# Classical Monte Carlo of $S_N$ Model on Hypertree Lattice

In [1]:
%reload_ext snakeviz
%run 'MC.py'

## Hypertree Model

### Model Setup
* Lattice: hypertree (branching MERA like, binary branching and binary merging)
* Degrees of freedom on each site is a permutation group element $g_i\in S_N$.
* Hamiltonian:
$$H = -\sum_{\langle ij\rangle} K_{ij}\chi(g_i,g_j),$$
where $\chi(g_i,g_j)=\text{Tr}g_i^{-1}g_j$ is the group character (the number of cycles).

In [23]:
lattice(4).sites

[<2,0,0>,
 <2,1,0>,
 <2,2,0>,
 <2,3,0>,
 <1,0,0>,
 <1,0,1>,
 <1,1,0>,
 <1,1,1>,
 <0,0,0>,
 <0,0,1>,
 <0,0,2>,
 <0,0,3>]

In [2]:
group(3).elements

[(0, 1, 2), (0, 2, 1), (1, 0, 2), (1, 2, 0), (2, 0, 1), (2, 1, 0)]

### Example
Generate model, run, get hist and energy, make measurements.
* use **block Gibbs sampling** + **heatbath update**, alternatively sampling on A and B sublattices.
 * A sublattice is the one that contains the most UV layer
 * B sublattice contains the rest
 * the boundary is labeled by C
* hist and magnetization only conts the bulk spins, boundary spins are not taken into account.

In [3]:
sys = hypertree_model(4,2,(1.,0.1)).run(100)
for k in range(10):
    sys.run(mode = 1)
    print(sys.config(),sys.hist(),sys.energy())
sys.measure(100000,monitor=[0,1,2,3])

[0 0 0 0 1 0 0 1 0 0 0 0] [6 2] 5.700000000000001
[0 1 1 1 0 0 1 1 0 0 0 0] [3 5] 1.900000000000001
[1 0 1 0 0 0 1 1 0 0 0 0] [4 4] 3.9000000000000012
[0 1 1 1 0 0 0 1 0 0 0 0] [4 4] 2.8000000000000016
[0 0 0 1 0 0 0 1 0 0 0 0] [6 2] 2.6000000000000014
[0 1 1 0 0 0 1 1 0 0 0 0] [4 4] 2.1000000000000014
[0 1 1 1 0 0 1 1 0 0 0 0] [3 5] 1.9000000000000015
[0 1 1 1 0 0 1 0 0 0 0 0] [4 4] 2.8000000000000016
[1 0 0 0 0 0 0 1 0 0 0 0] [6 2] 4.600000000000001
[0 0 1 1 0 0 1 1 0 0 0 0] [4 4] 1.7000000000000013


{'energy1': 2.8957269999997366,
 'energy2': 10.340216499997263,
 'magnet1': array([ 0.6682725,  0.3317275]),
 'magnet2': array([[ 0.4836075,  0.184665 ],
        [ 0.184665 ,  0.1470625]]),
 'spins': array([[ 0.81365,  0.54088,  0.54615,  0.50365],
        [ 0.18635,  0.45912,  0.45385,  0.49635]]),
 'steps': 100000}

The CPU intensive MC samplings are performed by the FORTRAN extension MC. The module MC contains two parts:
* core (for MC sampling and update macroscopic observibles)
* physics (for data collection and measurements)

In [3]:
import MC
print(MC.__doc__)

This module 'MC' is auto-generated with f2py (version:2).
Functions:
Fortran 90/95 modules:
  core --- irng,energy,nsite,config,chi,nc,nb,klst,na,hist,dof,jlst,init(),run(),get_energy(),get_hist(),dump(),load()  physics --- monitor,energy1,magn2,spins,magn1,energy2,measure().


### Profiling

In [6]:
%snakeviz hypertree_model(128,4,(1.,0.1)).run(1000)

 
*** Profile stats marshalled to file '/var/folders/lt/kgph6qbj6y306pfhbyd1kn2r0000gn/T/tmp9yqe_5z8'. 


In [8]:
def test():
    sys = hypertree_model(128,4,(1.,0.1))
    for k in range(1000):
        sys.run()
%snakeviz test()

 
*** Profile stats marshalled to file '/var/folders/lt/kgph6qbj6y306pfhbyd1kn2r0000gn/T/tmpip4uonm9'. 
